In [19]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
 """))

<span style="color:orange; font-size: 35px; font-weight: bold;"> ch15.데이터베이스 연동 </span></br>

# 1절. SQLite 데이터베이스 연동
- SQLite 데이터베이스는 별도의 DBMS없이 SQL을 이용하여 DB 엑세스 가능한 디스크 기반 DB (보안에 취약)
- SQLite는 프로토타입(시제품) 용도
- 프로젝트 진행 단계 : 분석 - 설계 - 구현 - 테스트 - 고객에게 배포 - 유지보수
-                   SQLite  |  Oracle/MySQL,MariaDB,mongoDB, ... 
- C 라이브러리 
- db browser for sqlite[https://sqlitebrowser.org/]
## 1.1 SQLite 패키지 load


In [5]:
import sqlite3
sqlite3. sqlite_version

'3.40.1'

In [7]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터베이스 연결 

In [8]:
# DB 연결 
conn = sqlite3.connect('data/ch15_exmple.db') #db파일을 읽으려면 db확장자로 
conn #non이면 연결이 안 된거이고 connection at 어쩌고 저쩌고면 연결됨

In [9]:
# 커서객체(SQL문 실행 -> 결과 조회) 생성
cursor = conn.cursor()
cursor

In [10]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE   INT,
        EMALE TEXT)
''')

###  insert,update,delete 전송

In [22]:
# sql문 전송 
cursor.execute('''
    INSERT INTO MEMBER VALUES('홍길동', 20, 'h@h.com')
''')
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1


In [23]:
sql = "INSERT INTO MEMBER VALUES('임길동', 20, 'h@h.com')"
print(sql)

INSERT INTO MEMBER VALUES('임길동', 20, 'h@h.com')


In [21]:
sql = "INSERT INTO MEMBER VALUES('박길동', 30, 'k@h.com')"
print(sql)
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)

INSERT INTO MEMBER VALUES('박길동', 30, 'k@h.com')
수행 결과 행수 : 1


In [24]:
conn.commit() # 반대는 conn.rollback() DML 에서만 가능

### select 전송

In [25]:
# SELECT문 전송
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMALE FROM MEMBER
''')


In [26]:
# SELECT 전송
#      seledt 문 실행결과를 받는 함수 
#      cursor.fetchone()   : 결과를 한행씩 받을 때 (튜플 홍길동, 21,h@h)
#      cursor.fetchall()   : 결과를 모두 받을 때 (튜플 list)
#      cursor.fetchmany(n) : 결과를 n행 받을 때 (튜플 list)
#      cursor.description  : header 내용을 포함한 내용들 (list)

# INSERT, UPDATE, DELETE 전송 : cursor.rowcont
print(cursor.fetchall())

[('홍길동', 21, 'h@h.com'), ('홍길동', 21, 'h@h.com'), ('김길동', 31, 'k@h.com'), ('박길동', 31, 'k@h.com'), ('홍길동', 21, 'h@h.com')]


In [27]:
print(cursor.fetchall())

[]


In [30]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMALE FROM MEMBER ORDER BY AGE
''')
members = cursor.fetchmany(3)
members

[('홍길동', 21, 'h@h.com'), ('홍길동', 21, 'h@h.com'), ('홍길동', 21, 'h@h.com')]

In [31]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMALE FROM MEMBER ORDER BY AGE
''')
members = cursor.fetchmany()
members

[('홍길동', 21, 'h@h.com')]

In [32]:
cursor.fetchall()

[('홍길동', 21, 'h@h.com'),
 ('홍길동', 21, 'h@h.com'),
 ('김길동', 31, 'k@h.com'),
 ('박길동', 31, 'k@h.com')]

In [45]:
# 데이터 한줄씩 가져오기

cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMALE FROM MEMBER ORDER BY AGE
''')
members =[]
while True:
    member = cursor.fetchone()
    if member is None:
        break
    print(member)
    members.append({'name':member[0], 'age':member[1], 'email':member[2]})
import pandas as pd
pd.DataFrame(members)

('홍길동', 21, 'h@h.com')
('홍길동', 21, 'h@h.com')
('홍길동', 21, 'h@h.com')
('김길동', 31, 'k@h.com')
('박길동', 31, 'k@h.com')


,name,age,email
0,홍길동,21,h@h.com
1,홍길동,21,h@h.com
2,홍길동,21,h@h.com
3,김길동,31,k@h.com
4,박길동,31,k@h.com


In [44]:

cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMALE FROM MEMBER ORDER BY AGE
''')

members = cursor.fetchall()
members
df = pd.DataFrame(members, columns=[description[0] for description in cursor.description ])
df

,NAME,NEXTAGE,EMALE
0,홍길동,21,h@h.com
1,홍길동,21,h@h.com
2,홍길동,21,h@h.com
3,김길동,31,k@h.com
4,박길동,31,k@h.com


In [42]:
# select문을 수행한 필드 정보 (해더 정보)
cursor.description

(('NAME', None, None, None, None, None, None),
 ('NEXTAGE', None, None, None, None, None, None),
 ('EMALE', None, None, None, None, None, None))

In [43]:
[description[0] for description in cursor.description ]

['NAME', 'NEXTAGE', 'EMALE']

## 1.3 SQL 구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우가 있음) : 오라클 불가 
- named(추천) 


In [47]:
conn = sqlite3.connect('data/ch15_exmple.db' )
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('박길동','김길동')")
cursor.fetchall()

[('김길동', 30, 'k@h.com'), ('박길동', 30, 'k@h.com')]

In [49]:
# 파라미터 사용하기 : qmark 방법 이용
sql = "SELECT * FROM MEMBER WHERE NAME IN (?,?)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른 이름은?')
cursor.execute(sql,(name1, name2))
cursor.fetchall()

검색할 이름은?성춘향
검색할 다른 이름은?김길동


[('김길동', 30, 'k@h.com')]

In [50]:
# 파라미터 사용하기 : named 방법 이용
sql = "SELECT * FROM MEMBER WHERE NAME IN (:name1,:name2)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른 이름은?')
cursor.execute(sql,{'name1':name1, 'name2':name2}) 
cursor.fetchall()

검색할 이름은?성춘향
검색할 다른 이름은?김길동


[('김길동', 30, 'k@h.com')]

In [51]:
# named 방법으로 데이터 입력하기 
try:
    name = input ('입력할 이름은?')
    age = int(input('입력할 나이는?'))
except ValueError:
    print('유효하지 않은 나이를 입력하시면 18세로 초기화 됩니다.')
    age = 18
finally:
    email = input('입력할 이메일은?')
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)',
               {'name':name,'age':age,'email':email})
conn.commit()
if cursor.rowcount == 1:
    print('입력 성공')
else:
    print('입력 실패')

입력할 이름은?함길동
입력할 나이는?ㅁ
유효하지 않은 나이를 입력하시면 18세로 초기화 됩니다.
입력할 이메일은?a@a.com
입력 성공


In [53]:
cursor.close()
conn.close()

# 2절. 오라클 데이터베이스 연결
- pip install cx_oracle

In [54]:
import cx_Oracle 
# conn 얻어오는 방법 1
conn = cx_Oracle.connect('scott','tiger','localhost:1521/xe')
conn
# conn = sqlite3.connect('파일')

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [55]:
# conn 얻어오는 방법 2
oracle_dsn = cx_Oracle.makedsn(host='localhost',port=1521,sid='xe')
conn = cx_Oracle.connect('scott','tiger',dsn=oracle_dsn)
conn


<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [57]:
cursor = conn.cursor()
cursor.execute('SELECT EMPNO NO, ENAME, JOB, MGR, SAL, COMM FROM EMP')
emps = cursor.fetchall()
for emp in emps:
    print(emp)
import pandas as pd
pd.DataFrame(emps, columns=[des[0] for des in cursor.description])

(7369, 'SMITH', 'CLERK', 7902, 800.0, None)
(7499, 'ALLEN', 'SALESMAN', 7698, 1600.0, 300.0)
(7521, 'WARD', 'SALESMAN', 7698, 1250.0, 500.0)
(7566, 'JONES', 'MANAGER', 7839, 2975.0, None)
(7654, 'MARTIN', 'SALESMAN', 7698, 1250.0, 1400.0)
(7698, 'BLAKE', 'MANAGER', 7839, 2850.0, None)
(7782, 'CLARK', 'MANAGER', 7839, 2450.0, None)
(7788, 'SCOTT', 'ANALYST', 7566, 3000.0, None)
(7839, 'KING', 'PRESIDENT', None, 5000.0, None)
(7844, 'TURNER', 'SALESMAN', 7698, 1500.0, 0.0)
(7876, 'ADAMS', 'CLERK', 7788, 1100.0, None)
(7900, 'JAMES', 'CLERK', 7698, 950.0, None)
(7902, 'FORD', 'ANALYST', 7566, 3000.0, None)
(7934, 'MILLER', 'CLERK', 7782, 1300.0, None)


,NO,ENAME,JOB,MGR,SAL,COMM
0,7369,SMITH,CLERK,7902.0,800.0,NaN
1,7499,ALLEN,SALESMAN,7698.0,1600.0,300.0
2,7521,WARD,SALESMAN,7698.0,1250.0,500.0
3,7566,JONES,MANAGER,7839.0,2975.0,NaN
4,7654,MARTIN,SALESMAN,7698.0,1250.0,1400.0
5,7698,BLAKE,MANAGER,7839.0,2850.0,NaN
6,7782,CLARK,MANAGER,7839.0,2450.0,NaN
7,7788,SCOTT,ANALYST,7566.0,3000.0,NaN
8,7839,KING,PRESIDENT,NaN,5000.0,NaN
9,7844,TURNER,SALESMAN,7698.0,1500.0,0.0
